# 検証用 Fact-Checker
チームみらいが公開しているFact-CheckerからAI関連の処理のみを抽出して、動作を検証しやすいようにいpythonで実行できるようにしました。  
Fact-Checkerをコマンドラインで動かすのと同様の動作で、入力されたテキストに対して判定を行います。  

## 準備
OpenAIのAPIキーに加えて、任意の参照したい資料をアップロードしたベクトルストアのIDを用意しておいてください。  
ベクトルストアについては、APIプラットフォームにログインして、Dashboard > Storage からアップロードするのが手軽かと思います。  
openaiのライブラリはあまり古すぎなければ実行可能かと思いますが、エラーとなるような場合はライブラリをアップデートしてください。  
以下で動かした際のバージョンは1.82.0です。

In [1]:
# %pip install openai

from openai import OpenAI

In [ ]:
# .envを使用してAPIキーを読み込む場合
# from dotenv import load_dotenv
# load_dotenv()

client = OpenAI()

# ベクトルストアのIDを指定
vectorStoreId = "your_vector_store_id"

In [ ]:
def fact_check(statement: str) -> str:
    """
    ファクトチェックを行う関数

    :param statement: チェック対象の文
    :return: ファクトチェックの結果
    """
    if not statement:
        return "❌ 文章を引数で渡してください。例:\n fact_check('地球は平らである')"


    response = client.responses.create(
        model= "o3-mini",
        tools=[{
            "type": "file_search",
            "vector_store_ids": [vectorStoreId],
        }],
        include=["file_search_call.results"],
        input=[
            {
                "role": "system",
			"content": """あなたはファクトチェッカーです。  
データソースと突き合わせて真偽を判定し、以下のフォーマットで答えてください。

────────────────────────────────
▼ステップ 0 : 事前フィルタ
  ❶ 入力テキストが「客観的で検証可能な事実命題」か判定せよ。
    ・感想／意見／価値判断のみ、あるいは一時的・主観的形容は対象外  
    ・データソースに対応項目がまったく存在しない場合も対象外
  ❷ 対象外なら OK とだけ出力し、理由を 1 行で補足（出典不要）。終了。

▼ステップ 1 : 真偽判定（ステップ 0 を通過した場合のみ）
  ❶ データソースで裏が取れるか確認し、次の三つから一つを選んで冒頭に出力  
      OK  : データソースと完全に一致  
      NG  : データソースと矛盾（誤りがある）  
      OK : データソースに十分な情報がなく判定不能
  ❷ その下に、判定根拠の短い説明  
  ❸ どの箇所に載っているか（節・ページ等）を箇条書き  
  ❹ 最後に出典（URL/文献名など）

▼フォーマット例
OK
- 根拠: …
- 該当箇所: …
- 出典: …

NG
- 誤り: …
- 正しい情報: …
- 出典: …

OK  ←ステップ 0 で終了
入力文は主観的感想であり客観的事実ではないため。
────────────────────────────────
        """,
                },
                {
                    "role": "user",
                    "content": statement,
                },
            ],
    )

    return response.output_text

## 検証例

おそらく検証用に使われたと思われる[ファイル](
https://github.com/team-mirai/fact-checker/blob/15718cf7926685f0c5f15ad2fbf056009fb7a2a6/manifest.md)も公開されていたので、このファイルをベクトルストアにアップロードして、主な動作を確認してみます。
Temperatureなどのパラメータの指定はされていないので、実行によって以下とは異なる回答・結果となる可能性はあります。

４つ目の「データソースに十分な情報がなく判定不能なためOK」のケースは、参照するファイルが１ファイルのみなことや聞き方にも影響されると思いますが、参照するファイルに記述がないということでNGと判定されることが多いかもしれません。

In [ ]:
 # 主観的な感想なのでOK
print(fact_check("今日は楽しかったです。") )

 # データソースと一致しているのでOK
print(fact_check("チームみらいのマニフェストは作成途中版らしい") )

 # データソースと矛盾しているのでNG
print(fact_check("チームみらいのマニュフェストは５月末でもう完成している") )

 # データソースに十分な情報がなく(検証用に使用したファイルには未記載)、判定不能なためOK (結果ではMGと判定)
print(fact_check("チームみらいはファクトチェック用のシステムをリリースしている") )

OK  
- 理由: この文章は個人の主観的な感想であり、客観的かつ検証可能な事実命題ではないため。
OK
- 根拠: アップロードされた manifest.md ファイル内に「本マニフェストは作成途中版」と明記されており、マニフェストがまだ作業中で変更の可能性があることが記述されています 。
- 該当箇所: manifest.md の冒頭部分および複数箇所（例: 、）で作成途中であることが明示されています。
- 出典: manifest.md (アップロードされたファイル)
NG
- 誤り: チームみらいのマニュフェストは「完成」ではなく、公開されているドキュメントには「作成途中版」であり、議論中・作業中の箇所が多いと記されているため、５月末に完成したとの記述と矛盾します。
- 正しい情報: マニフェストは皆さんの意見を取り入れながら改善される進行中のバージョンであり、完成版として発表されたものではありません。たとえば、manifest.md の内容には「作業中の箇所もたくさんある」との記述があります .
- 出典: manifest.md ()
NG
- 誤り: 提供された文書内には、チームみらいがファクトチェック用のシステムをリリースしたという記述は見当たりません。
- 正しい情報: マニフェストなどの文書は、政策や運営方針について論じられており、ファクトチェック用のシステムのリリースについては触れられていません  .
- 出典: manifest.md (複数箇所)
